## 2. Channel Allocation

In [1]:
import pandas as pd
data = pd.read_csv("subscribers.csv")
data.head()
subscriber_df = data.copy()
subscriber_df.head()

,Unnamed: 0,subid,package_type,num_weekly_services_utilized,preferred_genre,intended_use,weekly_consumption_hour,num_ideal_streaming_services,age,male_TF,...,join_fee,language,payment_type,num_trial_days,current_sub_TF,payment_period,last_payment,next_payment,cancel_date,trial_completed
0,162502,21724479,economy,NaN,comedy,access to exclusive content,NaN,NaN,NaN,False,...,NaN,ar,Standard Charter,14,True,1,2020-02-07 21:44:16,2020-06-07 21:44:16,NaN,True
1,380374,23383224,base,NaN,comedy,access to exclusive content,22.951579,NaN,70.0,False,...,0.3303,ar,NaN,14,True,1,2020-03-15 15:44:35,2020-07-15 15:44:35,NaN,True
2,496617,26844789,enhanced,3.0,regional,replace OTT,36.001186,2.0,25.0,True,...,0.1101,ar,NaN,14,False,0,NaN,NaN,NaN,True
3,356799,29417030,base,NaN,drama,replace OTT,20.051667,NaN,30.0,False,...,0.0367,ar,NaN,14,False,0,NaN,NaN,NaN,True
4,681701,26723159,base,4.0,comedy,replace OTT,22.951579,3.0,28.0,False,...,0.0367,ar,NaN,14,True,2,2020-02-19 12:57:07,2020-06-19 12:57:07,NaN,True


In [2]:
channel_spend_raw = pd.read_csv('channel_spend_graduate.csv', dtype=str)
channel_spend = channel_spend_raw.copy()
channel_spend.head()

,channel,date,spend_AED
0,bing,201907,400
1,display,201907,12
2,facebook,201907,9000
3,search,201907,13000
4,youtube,201907,90


In [3]:
channel_spend['year'] = 0
channel_spend['month'] = 0
channel_spend['sales'] = 0
channel_spend.head()

,channel,date,spend_AED,year,month,sales
0,bing,201907,400,0,0,0
1,display,201907,12,0,0,0
2,facebook,201907,9000,0,0,0
3,search,201907,13000,0,0,0
4,youtube,201907,90,0,0,0


In [4]:
subscriber_df['account_creation_date'] = pd.to_datetime(subscriber_df['account_creation_date'])
subscriber_df['account_creation_year'] = subscriber_df['account_creation_date'].map(lambda row: row.year)
subscriber_df['account_creation_month'] = subscriber_df['account_creation_date'].map(lambda row: row.month)

In [5]:
def find_sales(row):
    row_month = int(row['date'][-2:])
    row_year = int(row['date'][:4])
    row['month'] = row_month
    row['year'] = row_year
    row['sales'] = subscriber_df['subid'][(subscriber_df['account_creation_month'] == row_month)
                                         & (subscriber_df['account_creation_year'] == row_year)
                                         & (subscriber_df['attribution_technical'] == row['channel'])].count()
    return row

In [6]:
channel_spend = channel_spend.apply(find_sales, axis=1)
channel_spend.head()

,channel,date,spend_AED,year,month,sales
0,bing,201907,400,2019,7,120
1,display,201907,12,2019,7,44
2,facebook,201907,9000,2019,7,10011
3,search,201907,13000,2019,7,1328
4,youtube,201907,90,2019,7,8


In [7]:
channel_spend = channel_spend.sort_values(by=['channel', 'year', 'month']).set_index(['channel', 'date'])
channel_spend.head()

spend_AED  year  month  sales
channel date                                
bing    201907       400  2019      7    120
        201908       900  2019      8     71
        201909      1000  2019      9     84
        201910      1100  2019     10     86
        201911      1300  2019     11    103

In [8]:
channel_spend

spend_AED  year  month  sales
channel  date                                
bing     201907       400  2019      7    120
         201908       900  2019      8     71
         201909      1000  2019      9     84
         201910      1100  2019     10     86
         201911      1300  2019     11    103
         201912       300  2019     12     28
         202001      2100  2020      1    172
         202002      3700  2020      2    303
display  201907        12  2019      7     44
         201908        20  2019      8     74
         201909        13  2019      9     12
         201910        19  2019     10     50
         201911        29  2019     11     90
         201912        31  2019     12    112
         202001       148  2020      1    536
         202002        94  2020      2    341
facebook 201907      9000  2019      7  10011
         201908     13000  2019      8   9855
         201909     10500  2019      9   6129
         201910     17000  2019     10   9489
         201911     23000  2019     11  12995
         201912     16000  2019     12   9288
         202001     11000  2020      1   6289
         202002     14000  2020      2   7988
search   201907     13000  2019      7   1328
         201908     18500  2019      8   1939
         201909     19000  2019      9   1852
         201910     24000  2019     10   2333
         201911     25000  2019     11   2510
         201912     38000  2019     12   3770
         202001     41000  2020      1   4073
         202002     44000  2020      2   4300
youtube  201907        90  2019      7      8
         201908       180  2019      8     15
         201909       100  2019      9      2
         201910       130  2019     10     11
         201911       550  2019     11     47
         201912       900  2019     12     76
         202001      4360  2020      1    370
         202002      2420  2020      2    205